In [1]:
%run package_import.ipynb
%run data_profiling.ipynb

In [2]:
from sqlalchemy import create_engine, types
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [59]:
df_combined = pd.read_sql_query(

''' 
select *,
    'train' as source
from raw_train

union all

select *, 
    cast(null as float) as SalePrice,
    'test' as source
from raw_test

'''
,dbConnection)

In [60]:
df_profile = main_data_profiling(df_combined)

In [82]:
df_profile

,column_names,col_desc,value_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,NaN,nan,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,Identifies the type of dwelling involved in the sale.,"{'20': '1-STORY 1946 & NEWER ALL STYLES', '30': '1-STORY 1945 & OLDER', '40': '1-STORY W/FINISHED ATTIC ALL AGES', '45': '1-1/2 STORY - UNFINISHED ALL AGES', '50': '1-1/2 STORY FINISHED ALL AGES', '60': '2-STORY 1946 & NEWER', '70': '2-STORY 1945 & OLDER', '75': '2-1/2 STORY ALL AGES', '80': 'SPLIT OR MULTI-LEVEL', '85': 'SPLIT FOYER', '90': 'DUPLEX - ALL STYLES AND AGES', '120': '1-STORY PUD (Planned Unit Development) - 1946 & NEWER', '150': '1-1/2 STORY PUD - ALL AGES', '160': '2-STORY PUD - 1946 & NEWER', '180': 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', '190': '2 FAMILY CONVERSION - ALL STYLES AND AGES'}",int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,Identifies the general zoning classification of the sale.,"{'A': 'Agriculture', 'C': 'Commercial', 'FV': 'Floating Village Residential', 'I': 'Industrial', 'RH': 'Residential High Density', 'RL': 'Residential Low Density', 'RP': 'Residential Low Density Park', 'RM': 'Residential Medium Density'}",object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,Linear feet of street connected to property,{},float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,Lot size in square feet,{},int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,Type of road access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved'}",object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,Type of alley access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved', 'NA': 'No alley access'}",object,3,NaN,NaN,NaN,NaN,2721,2919,0.932169,NaN,NaN
7,LotShape,General shape of property,"{'Reg': 'Regular', 'IR1': 'Slightly irregular', 'IR2': 'Moderately Irregular', 'IR3': 'Irregular'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,Flatness of the property,"{'Lvl': 'Near Flat/Level', 'Bnk': 'Banked - Quick and significant rise from street grade to building', 'HLS': 'Hillside - Significant slope from side to side', 'Low': 'Depression'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,Type of utilities available,"{'AllPub': 'All public Utilities (E,G,W,& S)', 'NoSewr': 'Electricity, Gas, and Water (Septic Tank)', 'NoSeWa': 'Electricity and Gas Only', 'ELO': 'Electricity only'}",object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


### ETL Process

In [61]:
# The loop is to convert description to jason to fill na value
# initalise the dictionary
desc_dict, attr_dict, col_desc = {}, {}, {}
with open('House_Prices/data_description.txt', encoding='utf8') as f:
    for line in f:
        # convert line into list
        words = line.split()
        # if the line contains ":", the line includes a column
        # and its description
        if words and ':' in line:
            # convert the line into list, recognise the first item from the list
            # as the column and remove ":"
            col = words[0].replace(':','')
            # save the column as a key into description dictionary
            desc_dict[col] = {}
            col_desc[col] = ' '.join(words[1:])
        # if the line is blank, move to the next one
        elif  not words:
            continue
        else:
            # the attr_desc is to contain values for each column
            # concatenate items except the value from the list to 
            # rebuild the description of each value
            attr_desc = ' '.join(words[1:])
            # store the first item of the list as the value
            attr = words[0]
            # save the the attribute dictionary as sub-dictionary to
            # description description
            desc_dict[col][attr] = attr_desc

In [62]:
df_profile = main_data_profiling(df_combined)
df_profile.insert(1, 'col_desc', np.nan)
df_profile.insert(2, 'value_desc', np.nan)
df_profile['col_desc'] = df_profile['column_names'].map(col_desc)
df_profile['value_desc'] = df_profile['column_names'].map(desc_dict)
df_profile['value_desc'] = df_profile['value_desc'].astype('str')

#### 1. Replace NA

In [66]:
df_combined['SaleType'] = df_combined['SaleType'].fillna('WD')
df_combined['MSZoning'] = df_combined['MSZoning'].fillna('RL')
df_combined['Functional'] = df_combined['Functional'].fillna('Typ')
df_combined['Utilities'] = df_combined['Utilities'].fillna('AllPub')
df_combined['Electrical'] = df_combined['Electrical'].fillna('SBrkr')
df_combined['KitchenQual'] = df_combined['KitchenQual'].fillna('TA')
df_combined['Exterior2nd'] = df_combined['Exterior2nd'].fillna('VinylSd')
df_combined['Exterior1st'] = df_combined['Exterior1st'].fillna('VinylSd')

# the loop is to fill na value based on descriptioin automatically
for col in df_combined.columns:
    # if a column has null value and its value includes "NA"
    # replace NA with the value mentioned in the description;
    # otherwise, skip it. 
    if df_profile[df_profile['column_names'] == col]['null_values'].squeeze() > 0:
        if df_profile[df_profile['column_names'] == col]['data_type'].squeeze() == 'object':
                    df_combined[col] = df_combined[col].fillna('No')
        elif df_profile[df_profile['column_names'] == col]['data_type'].squeeze() == 'float64':
            df_combined[col] = df_combined[col].fillna(0)

#### 2. Check duplication

In [127]:
df_combined[df_combined.duplicated()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,source


#### map rates for quality

In [80]:
marks = {"No":-1, "Po": 0, 'Fa': 1, "TA": 2, 'Gd': 3, 'Ex': 4}
cols_marks = []

for col in df_combined.columns:
    marks_exist = 1
    for i in df_combined[col].unique():
        if i in marks:
            pass
        else:
            marks_exist = 0
            break
    
    if marks_exist == 1:
        cols_marks.append(col)


for col in cols_marks:
    df_combined[col] = df_combined[col].map(marks)

#### Feature engineer

In [81]:
df_combined['BsmtFinSF'] = df_combined['BsmtFinSF1'] + df_combined['BsmtFinSF2']
df_combined['Overall'] = df_combine['']

array([-1,  4,  1,  3])

### Upload transformed data

In [141]:
df_combined.to_sql('transformed_combined', dbConnection, if_exists='replace', index=False)

In [32]:
df_profile.to_sql('raw_data_profile', dbConnection, if_exists='replace', index=False, method='multi')